# Using ArcGIS API for Python

## What is the Python API for ArcGIS?

- Manages your Web GIS
    + AKA: your ArcGIS Enterprise, ArcGIS Online
- Collection of Pythonic Tools to managing your web presence 

### What's Inside?

<img src="./img/guide_api_overview_modules.png" width=30%/>

- tools to managing your GIS 
- tool to perform spatial analysis
- tool managing your content
- tools for managing others content

## Getting Started

<img src="./img/catchingfly.jpg"/>

"Man who catch fly with chopstick accomplish anything"
~Miyagi

### Import This!

In [ ]:
from arcgis.gis import GIS

### Authentication Schemes

#### Anonymous users

In [ ]:
gis = GIS() 

#### built-in accounts

In [ ]:
gis = GIS(username='andrew57')

In [ ]:
gis.users.me

#### Using Pro Authentication

In [ ]:
gis = GIS('pro')
gis.properties.isPortal, gis.url

#### Other Formats:

- Active Directory
- PKI

### Creating Profiles

- Allows for securing username/password
- Leverages Operating System's password manager

In [ ]:
import getpass
gis = GIS(username='andrew57', password=getpass.getpass(), profile="agol_profile_demo")
del gis

In [ ]:
gis = GIS(profile='agol_profile_demo')
gis.users.me

### Handling SAML

- SAML or two-factor authentication you **MUST** leverage ArcGIS Pro OR Oauth 

## The GIS Object

<img src='./img/window.png' width=50% />

- This is the entry class to any Web GIS!
- From this object you access data, manage systems and users


### Properties

- Access the organization's  `properties`
- It includes information such as:
    + the name
    + logo
    + featured items
    + supported protocols


#### Accessing Properties



In [ ]:
gis = GIS(profile='your_online_profile')

In [ ]:
gis.properties

In [ ]:
gis.properties.availableCredits

#### Updating Properties

In [ ]:
gis.update_properties(
    {
        'name' : "New Name"
    }
)

In [ ]:
gis.properties.name

In [ ]:
gis.update_properties(
    {
        'name' : 'Developer Site For Geosaurus'
    }
)

### Manager Classes

<img src="https://developers.arcgis.com/assets/img/python-graphics/guide_gis_module_01.png" />

### Visualizing Data

- The GIS object provide a web map integration for Jupyter Notebooks

In [ ]:
gis = GIS()

In [ ]:
m = gis.map("London, England")
m.basemap = 'dark-gray-vector'
m

In [ ]:
item = gis.content.get("7f464b9c7682420c9b093ad6e5cae04f")
item

In [ ]:
m.add_layer(
    {"type":"FeatureLayer", 
      "url":item.layers[0].url,
      "renderer":"ClassedColorRenderer",
      "field_name":"F_born_osea",
      "opacity":0.7}
    )

### Version

- It's always good to know what version of the REST API you are using

In [ ]:
gis = GIS(profile='your_online_profile')
gis.version

## Managing Content

<img src="./img/managing_content.png" />

### Searching

- `search` provides a way to access shared content
- Robust method that can limit by:
    + keyword
    + item type
    + sort results
    + limit number of return items


In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile')

- Accessing the Help

In [ ]:
gis.content.search?

**Limiting Searches to Inside the Organization**

In [ ]:
items = gis.content.search("*", item_type="Feature Layer", max_items=10000, outside_org=False)
print("Number of Items: {n}".format(n=len(items)))

**Limiting Searches to Inside and Outside the Organization**

In [ ]:
items = gis.content.search("*", item_type="Feature Layer", max_items=10000, outside_org=True)
print("Number of Items: {n}".format(n=len(items)))

In [ ]:
items = gis.content.search("title:census", outside_org=True, max_items=10000)
print("Number of Items: {n}".format(n=len(items)))

    + there is no way to specify return all items

**Limiting Searches by Item Type**

In [ ]:
items = gis.content.search("*", item_type="Image Service", max_items=1000, outside_org=False)
print("Number of Items: {n}".format(n=len(items)))

##### Admin Task: How can we find out how many public items were created and shared publicly yesterday on AGOL?


In [ ]:
import datetime 
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days=1)
today = int(datetime.datetime.timestamp(today)) * 1000
yesterday = int(datetime.datetime.timestamp(yesterday)) * 1000
count = GIS().content.advanced_search("created: [{yesterday} TO {today}]".format(yesterday=yesterday, today=today), 
                             return_count=True)
print("Number of Items: {n}".format(n=count))

### Adding, Deleting, Publishing, and Updating Content

- Content drives a GIS system
    + Provides users a place to collaborate and share information
- Only users with publishing rights can create services
- Administrators can managing anyone's content

In [ ]:
type(gis.content)

#### Adding Content

- `add()` uploads content to a GIS
- Returns an `Item`
- Item Properties - information about the item
    + title, type and tags are always **required**

In [ ]:
gis = GIS(profile='your_online_profile')

In [ ]:
item = gis.content.add(item_properties={
        "tags" : ["Tiger Census", "Counties", "FGDB"],
        "type" : "File Geodatabase",
        "title" : "Census County Featureclass"
        },
        data=r"./data/counties_demo2.zip")
item

#### Updating Items

- Allows for the modification of all `Item` properties
- Add metadata and/or thumbnails


In [ ]:
item.update(thumbnail='./data/tiger_thumbnail.jpg')
item

#### Publishing Items

- Certain `Item` types can be published.
    + sd, Feature Classes, csv, etc...
- This creates a Hosted Feature Layer, Vector Tile, Tile Service

In [ ]:
item.publish?

In [ ]:
pitem = item.publish()
pitem

**Notice that the thumbnail and other information is past along the workflow**

In [ ]:
m = gis.map("United States")
m

In [ ]:
m.extent = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
 'xmin': -15508693.373344794,
 'ymin': 2521206.203338205,
 'xmax': -5959568.303736833,
 'ymax': 6434782.051538189}
m.add_layer(pitem)

### Protecting and Sharing

- Provides the ability to disseminate data with `sharing`
- Protecting ensure content does not get erased!
    + There is **NO** undo button on AGOL/Enterprise

#### Protecting a Single Item

In [ ]:
pitem.protect(enable=True)

In [ ]:
pitem.delete()

In [ ]:
pitem.protect(enable=False)
pitem.delete()

#### Sharing Content

Check the Item's sharing status:

In [ ]:
item.access

In [ ]:
item.share(everyone=True)
item.access

In [ ]:
item.share(everyone=False)
item.access

In [ ]:
item.shared_with

In [ ]:
item.delete()

### Creating Empty Services

- Create services without data or an item to publish from
- Fully customize schema via JSON

**1. Check if Service Name is Free**

In [ ]:
gis.content.is_service_name_available("DevSummit2021", service_type="featureService")

**2. Create an Empty Feature Service**

In [ ]:
item = gis.content.create_service(name="DevSummit2021")
item

**3. Create a New Layer**

In [ ]:
item.layers

In [ ]:
# Get the FS and the Manager
from arcgis.features import FeatureLayerCollection
fs = FeatureLayerCollection.fromitem(item)
mgr = fs.manager

In [ ]:
mgr.properties

In [ ]:
import json
with open("./data/layer.json", 'r') as reader:
    print(mgr.add_to_definition(json_dict=json.loads(reader.read())))
    

In [ ]:
fs.layers

In [ ]:
item.delete()

### Cloning Content

<img src="./img/cloning.jpg" />

- Allows for the copying of Hosted Feature Layers and items from one site to another
- **WILL NOT** configure applications

**`clone_items` provides the ability to copy content from one site to another**

<img src="./img/clone_items.jpg" />

## Working with Users

<img src="./img/usermanagement.jpg" />

- The GIS object contains functionality to create, manage, and monitor users

### Simple User Reports: `counts()` 

- simple report on the number of licenses currently used

In [ ]:
gis = GIS(profile='your_online_profile')

In [ ]:
gis.users.counts?

In [ ]:
df = gis.users.counts(type="user_type")
df

In [ ]:
gis.users.counts(type='roles')

**Getting Total User Count**

In [ ]:
df = gis.users.counts(type="user_type")
df['count'].sum()

### Creating User

- `gis.users.create()` provides a seamless operation to generate new users

In [ ]:
import uuid

**Example: Creating a Content Creator**

In [ ]:
user = gis.users.create(username=f"del_{uuid.uuid4().hex[:5]}",
                password='Sch0II4r0ckslap',
                firstname="General",
                lastname="Account",
                email="gac@esri.com",
                user_type='creatorUT')
user

In [ ]:
user.reset(new_security_question=1,
           new_security_answer="Redlands",
           password='Sch0II4r0ckslap',
           new_password="IMgoing2LA")

#### Remove the User

In [ ]:
l = gis.admin.license

In [ ]:
user.delete()

### Updating Users

- Like `Item` classes, users have `update()`


#### Updating User Properties

In [ ]:
gis = GIS(profile='your_online_profile')

In [ ]:
user = gis.users.create(username=f"del_{uuid.uuid4().hex[:5]}",
                password='dnc1ng1ntheDARK',
                firstname="Unknown",
                lastname="Account",
                email="gac@esri.com",
                role="iAAAAAAAAAAAAAAA",
                user_type="advancedUT")

In [ ]:
user

In [ ]:
user.update(first_name="Bruce", last_name="Springsteen", thumbnail="./data/user_thumbnail.jpg")
user

**Question: How do we let Bruce create content?**

In [ ]:
print((user.userLicenseTypeId, user.role))
user.update_license_type(user_type="creatorUT")
user.update_role("org_publisher")
print((user.userLicenseTypeId, user.role))

##### Clean up items.

In [ ]:
check_items = gis.content.search("rivers.gdb.zip")
for ditem in check_items:
    ditem.delete()
check_items = gis.content.search("US_Rivers", "Feature Layer")
for ditem in check_items:
    ditem.delete()

##### **Question: How do you add content to a user?**

In [ ]:
fp = "./data/rivers.gdb.zip"
item_properties = {
    'title' : "US_Rivers",
    'type' : 'File Geodatabase',
    'tags' : ['Rivers']
}
river_item = gis.content.add(data=fp, item_properties=item_properties, owner=user.username)

In [ ]:
river_item

In [ ]:
p_river = river_item.publish({'name' : f'bdata_{uuid.uuid4().hex[:4]}'})
p_river

In [ ]:
p_river.delete()
river_item.delete()

### Searching for Users

- `search` provides the ability to find users
- Locates both system and user accounts

In [ ]:
users = gis.users.search("del_*")
users

### List a User's Content

In [ ]:
items = users[0].items()
items

### Reassigning Content

In [ ]:
for item in users[0].items():
    print(item.reassign_to(target_owner=gis.users.me.username))

### Deleting Users

In [ ]:

user.delete()

In [ ]:
gis.users.search("del_*")

## Group Manager

### The Group

<img src="./img/groups.jpeg" />

- `GroupManager` provides tools to manage groups
    + create
    + update
    + delete
    + management content

In [ ]:
type(gis.groups)

### Searching for Groups

In [ ]:
agol_gis = GIS(profile='your_online_profile')

In [ ]:
agol_gis.groups.search(query="Dev*")

### Managing Groups

#### Creating a Group

In [ ]:
group = gis.groups.create(title="EuroDevDemoGroup",
                  tags=['demo', 'eraseme'], 
                  is_invitation_only=False,
                  auto_join=True)
group

In [ ]:
group.update(thumbnail=r"./data/group-icon.png")
group

#### Assigning Items to Group

In [ ]:
item = gis.users.me.items()[0]
item.share(groups=[group])

In [ ]:
item.shared_with

#### Adding Users to Group

In [ ]:
group.add_users(usernames=gis.users.search("*"))

#### Deleting a Group

In [ ]:
group.delete()